In [1]:

import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
%matplotlib inline

file_path = '/Users/haowei/Documents/gal/Day37_case_study2/case_study2/Churn/churn_train.csv'
def import_data(file_path):
    df = pd.read_csv(file_path)
    return df

def add_churn(df):
    df['last_trip_date'] = pd.to_datetime(df['last_trip_date'])
    condition = df['last_trip_date'] < '2014-06-01'
    df['churn'] = 1
    df.ix[~condition, 'churn'] = 0
    return df

def add_dummy(df, target_col):
    d = pd.get_dummies(df[target_col])
    d = d.drop(d.columns[-1], axis = 1)
    df_list = [df,d]
    df = pd.concat(df_list,axis =1)
    df = df.drop(target_col, axis =1)
    return df

def add_dummies(df, target_cols):
    dx = df.copy()
    for target_col in target_cols:
        dx = add_dummy(dx, target_col)
    return dx

df = import_data(file_path)
df = add_churn(df)

df_c = add_dummy(df, 'city')
cat_list = ['phone','city',]
df_c = add_dummies(df,cat_list)

def drop_column_nan(df, target_col):
    df = df[~df[target_col].isnull()]
    return df
dfr = drop_column_nan(df,'avg_rating_of_driver')
dfr.shape
# r = search_rate(df_drop, 0.7, 'avg_rating_of_driver' )
# print r
#df_c.head()

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


(33472, 13)

In [2]:
def crossVal(x_train, y_train, model, fold =5 ):
    #x_train, y_train should be numpy arrary
    # modle is a test modle, e.g., #sklearn.linear_model.LinearRegression()
    kf = sklearn.cross_validation.KFold(len(x_train), fold) 
    mses = []
    for train_ind, test_ind in kf:
        train_x = x_train[train_ind]
        train_y = y_train[train_ind]
        test_x = x_train[test_ind] # for the current q, we only use x_train for KFold
        test_y = y_train[test_ind]
        
        md = model.fit(train_x, train_y) # sklearn.linear_model.LinearRegression().fit(train_x, train_y)
        y_hat = md.predict(test_x)
        mse = sklearn.metrics.mean_squared_error(test_y, y_hat)
        mses.append(mse)
    return np.mean(mses)

churn_rate = df[df['churn']==1].shape[0]/float(df.shape[0])

df0 = df[df['churn'] ==1]
df1 = df[df['avg_rating_of_driver'].isnull()]
df2 = df1[df1.churn == 1]
print 'churned ',df0.shape[0]
print 'churn rate', churn_rate
print 'missing avg_rating_of_driver',df1.shape[0]
print 'missing avg_rating_of_driver and churned', df2.shape[0]
print 'churn rate for missing avg_rating_of_driver', df2.shape[0]/float(df1.shape[0])

df3 = df[df['avg_rating_of_driver'] < 0.5*df['avg_rating_of_driver'].mean()]
df4 = df3[df3.churn == 1]

df5 = df[df['avg_rating_of_driver'] >= df['avg_rating_of_driver'].mean()]
df6 = df5[df5.churn == 1]

df7 = df[df['avg_rating_by_driver'].isnull()]
df8 = df7[df7.churn == 1]

print df3.shape[0]
print df4.shape[0]
print df4.shape[0]/float(df3.shape[0])

print df5.shape[0]
print df6.shape[0]
print df6.shape[0]/float(df5.shape[0])

print 'missing avg_rating_by_driver',df7.shape[0]
print 'missing avg_rating_by_driver and churned',df8.shape[0]
print 'churn rate for missing avg_rating_by_driver',df8.shape[0]/float(df7.shape[0])
df.describe()

churned  24968
churn rate 0.6242
missing avg_rating_of_driver 6528
missing avg_rating_of_driver and churned 5197
churn rate for missing avg_rating_of_driver 0.796109068627
390
310
0.794871794872
21504
13154
0.61170014881
missing avg_rating_by_driver 162
missing avg_rating_by_driver and churned 130
churn rate for missing avg_rating_by_driver 0.802469135802


/Users/haowei/anaconda/envs/py27/lib/python2.7/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,surge_pct,trips_in_first_30_days,weekday_pct,churn
count,40000.000000,39838.000000,33472.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,5.791302,4.777434,4.601697,1.074956,8.857342,2.280700,60.874382,0.624200
std,5.708056,0.448088,0.614810,0.222427,20.014008,3.811289,37.089619,0.484335
min,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.420000,NaN,NaN,1.000000,0.000000,0.000000,33.300000,0.000000
50%,3.880000,NaN,NaN,1.000000,0.000000,1.000000,66.700000,1.000000
75%,6.930000,NaN,NaN,1.050000,8.300000,3.000000,100.000000,1.000000
max,160.960000,5.000000,5.000000,8.000000,100.000000,125.000000,100.000000,1.000000


In [27]:
# plan A: creat new cateroty column rating_cat where NaN rating and 
#ratings<0.5*mean will be assigned to 0, ratings>0.5*mean will be 1

def add_rating_cat(df):
    condition1 = df['avg_rating_of_driver'].isnull() 
    condition2 = df['avg_rating_of_driver'] < 0.5*df['avg_rating_of_driver'].mean()
    df['rating_category'] = 1
    df.ix[condition1|condition2, 'rating_category'] = 0
    df.drop('avg_rating_of_driver', axis = 1, inplace =True)
    return df

def impute_rating(df, target_col, value):
    condition = df[target_col].isnull() 
    df.ix[condition, target_col] = value
    return df

churn_rate = df[df['churn']==1].shape[0]/float(df.shape[0])

def search_optimal_impute_value(df, y, target_category, response):
    optional_values = sorted(df[target_category].unique(), reverse = True)
    for value in optional_values:       
        d = df[df[target_category] <= value]
        d_positive = d[d[response] ==1].shape[0]*1.0/d.shape[0]
        #print value, d_positive
        if d_positive >=y:
            return value
    else:
        return df[target_category].min() # will impute the min value
'''Make sure the missing value is removed from the target category!!!'''
r = search_optimal_impute_value(dfr, 0.79, 'avg_rating_of_driver','churn' )
print r
    

5.0 0.590672801147
4.9 0.490796093519
4.8 0.515513897867
4.7 0.539343292412
4.6 0.552891042781
4.5 0.571544862518
4.4 0.57304855224
4.3 0.590374331551
4.2 0.610933814347
4.1 0.629569093611
4.0 0.646903205727
3.9 0.577945520184
3.8 0.597753597754
3.7 0.618952419032
3.6 0.632486388385
3.5 0.651963160446
3.4 0.665074626866
3.3 0.680503144654
3.2 0.703366058906
3.1 0.715417867435
3.0 0.728550295858
2.9 0.71609403255
2.8 0.726432532348
2.7 0.737451737452
2.6 0.755693581781
2.5 0.756871035941
2.4 0.793367346939
2.4


In [ ]:
list_1 = ['churn','weekday_pct','avg_dist','avg_rating_by_driver','avg_rating_of_driver','avg_surge','surge_pct','trips_in_first_30_days']
list_2 = ['weekday_pct','avg_dist','surge_pct','trips_in_first_30_days','avg_surge']
list_3 = ['churn','avg_dist','avg_rating_by_driver','avg_rating_of_driver']

df_plot = df[list_3]
ax = scatter_matrix(df_plot, diagonal = 'kde', figsize = (8,8), alpha = 0.4)
plt.show()

x1 = df_new1['avg_rating_by_driver']
y1 = df_new1['avg_rating_of_driver']
c1 = np.array(df_new1['churn'])

plt.scatter(x1,y1, c = c1, alpha = 0.5)
plt.ylabel('avg_rating_of_driver')
plt.xlabel('avg_rating_of_driver')